In [ ]:
import os
os.chdir('../')
%pwd

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [ ]:
from src.coccidiosis_chicken_disease_classifier.constants import *
from src.coccidiosis_chicken_disease_classifier.utils.common import (
    read_yaml,
    create_directories,
)


class PrepareBaseModelConfigManager:
    def __init__(
        self, config_file_path=CONFIG_FILE_PATH, parmas_file_path=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories(self.config.artifacts_root)

    def get_prepare_base_model(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories(config.root_dir)

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_classes=self.params.CLASSES,
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_learning_rate=self.params.LEARNING_RATE,
            params_weights=self.params.WEIGHTS,
        )

        return prepare_base_model_config

In [6]:
import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf

/Users/vishalsharma/Downloads/Coccidiosis Chicken Disease Classifier/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
from src.coccidiosis_chicken_disease_classifier import logger


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        try:
            logger.info("Getting base model")

            self.model = tf.keras.applications.vgg16.VGG16(
                input_shape=self.config.params_image_size,
                weights=self.config.params_weights,
                include_top=self.config.params_include_top,
            )
            self.save_model(path=self.config.base_model_path, model=self.model)

        except Exception as e:
            raise e

    def update_base_model(self):
        self.full_model = self._prepare_base_model(
            model=self.model,
            freeze_all=True,
            freeze_till=None,
            classes=self.config.params_classes,
            learning_rate=self.config.params_learning_rate,
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def _prepare_base_model(model, freeze_all, freeze_till, classes, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:freeze_till]:
                model.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)
        predictions = tf.keras.layers.Dense(units=classes, activation="softmax")(
            flatten_in
        )

        full_model = tf.keras.models.Model(inputs=model.input, outputs=predictions)

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["acuracy"],
        )
        full_model.summary()
        return full_model
    
    @staticmethod
    def save_model(path: Path, model : tf.keras.model):
        model.save(path)
